In [10]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping


    
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        print(v)
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)
    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

def auc_roc_ignorante(y_true, y_pred):
    # any tensorflow metric
    value, _ = tf.metrics.auc(y_true,y_pred)
    return value

In [11]:
# generation a small dataset
N_all = 1000
N_tr = int(0.7 * N_all)
N_te = N_all - N_tr

np.random.seed(0)
x_dim = 50
y_dim = 30
X = np.random.rand(N_all,x_dim)
y = np.random.randint(0,2,size=(N_all-1,y_dim))
y_app = np.zeros(shape= (1,y_dim))
y_app[0] = 1
y = np.vstack((y,y_app))

X_train, X_valid = X[:N_tr, :], X[N_tr:, :]
y_train, y_valid = y[:N_tr, :], y[N_tr:, :]

In [ ]:
class MyCallBack(keras.callbacks.Callback):
    def __init__(self, callbacks,model):
            super().__init__()
            self.callback = callbacks
            self.model = model
            self.model_original = model
            self.aucs = []

    def on_epoch_begin(self,epoch,logs=None):
            self.model = self.model_original
            self.callback.on_epoch_begin(epoch, logs=logs)

    def on_epoch_end(self,epoch,logs=None):
            self.model = self.model_original
            y_pred = self.model.predict(self.validation_data)
            
            self.callback.on_epoch_end(epoch, logs=logs)

    def on_batch_end(self, batch, logs=None):
            self.model = self.model_original
            self.callback.on_batch_end(batch, logs=logs)

    def on_batch_begin(self, batch, logs=None):
            self.model = self.model_original
            self.callback.on_batch_begin(batch, logs=logs)

    def on_train_begin(self, logs=None):
            self.model = self.model_original
            self.callback.set_model(self.model)
            self.callback.on_train_begin(logs=logs)

    def on_train_end(self, logs=None):
            self.model = self.model_original
            self.callback.on_train_end(logs=logs)

cbk_tb = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, batch_size=batch_size, write_graph=True,
                                         write_grads=False, write_images=False, embeddings_freq=0,
                                         embeddings_layer_names=None, embeddings_metadata=None)

cbk_es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',
                                          min_delta=min_improvement, patience=patience, verbose=1)

cbk_mc = keras.callbacks.ModelCheckpoint(monitor='val_loss', mode='max', save_best_only=True, 
                                            filepath=checkpoint_dir+checkpoint_file_name, 
                                            verbose=1)

cbk = MyCallBack(cbk_tb, model)
cbk1 = MyCallBack(cbk_es, model)
cbk2 = MyCallBack(cbk_mc, model)

callbacks = [cbk,cbk1,cbk2]

In [12]:
# model & train
model = Sequential()
model.add(Dense(y_dim,input_shape=(X.shape[1],)))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', auc_roc])

my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

<tf.Variable 'metrics/auc_roc/auc/true_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics/auc_roc/auc/false_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics/auc_roc/auc/true_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics/auc_roc/auc/false_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/true_positives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/false_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/true_negatives:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'metrics_1/auc_roc/auc/false_positives:0' shape=(200,) dtype=float32_ref>


In [20]:
model.fit(X, y,batch_size=len(X), epochs=20, verbose=1,callbacks=my_callbacks)

# # or use independent valid set
# model.fit(X_train, y_train,
#           validation_data=(X_valid, y_valid),
#           batch_size=32, nb_epoch=5, verbose=1,
#           callbacks=my_callbacks)

Epoch 1/20
1000/1000 [==============================] - 0s 11us/step - loss: 0.7026 - acc: 0.5039 - auc_roc: 0.5003
Epoch 2/20
1000/1000 [==============================] - 0s 10us/step - loss: 0.7025 - acc: 0.5037 - auc_roc: 0.5004
Epoch 3/20
1000/1000 [==============================] - 0s 10us/step - loss: 0.7025 - acc: 0.5038 - auc_roc: 0.5006
Epoch 4/20
1000/1000 [==============================] - 0s 10us/step - loss: 0.7024 - acc: 0.5042 - auc_roc: 0.5007
Epoch 5/20
1000/1000 [==============================] - 0s 11us/step - loss: 0.7023 - acc: 0.5045 - auc_roc: 0.5009
Epoch 6/20
1000/1000 [==============================] - 0s 10us/step - loss: 0.7022 - acc: 0.5046 - auc_roc: 0.5010
Epoch 7/20
1000/1000 [==============================] - 0s 10us/step - loss: 0.7022 - acc: 0.5046 - auc_roc: 0.5011
Epoch 8/20
1000/1000 [==============================] - 0s 10us/step - loss: 0.7021 - acc: 0.5049 - auc_roc: 0.5013
Epoch 9/20
1000/1000 [==============================] - 0s 10us/step - l

In [21]:
y_pred = model.predict(X_train)

In [22]:
aucs = []
for i in range(y.shape[1]):
    fpr, tpr, thresholds = roc_curve(y_train[:,i], y_pred[:,i],)
    aucs.append(auc(fpr,tpr))
    print(aucs[i])
np.mean(aucs)

0.47728682930015354
0.4710925496093367
0.5237768576818482
0.5193266883811297
0.5193231378517482
0.5344299108313664
0.49582802651774927
0.5495099003409388
0.5488306767703163
0.47643654233912114
0.5437976506004385
0.48183956101384895
0.5
0.4935701163502756
0.5117184630078613
0.5229254433232096
0.48225195204024957
0.4748756645515349
0.5006043973994576
0.5145068488672399
0.49666964336919034
0.4864366239724406
0.5196749148917064
0.5295337972491752
0.5081209150326798
0.5006082079499719
0.47225309751871586
0.4927450980392156
0.5354842396584238
0.5053704067275285


0.5062942720395623

In [23]:
roc_auc_score(y_train, y_pred)

0.5062942720395623